<a href="https://colab.research.google.com/github/Himanshud27/LP2/blob/main/Assignment%2011-13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile wordcount.py
from mrjob.job import MRJob
import re

class wordc(MRJob):
    def mapper(self, _, line):
        words = re.findall(r'\w+', line.lower())
        for word in words:
            yield (word, 1)

    def reducer(self, key, values):
        yield (key, sum(values))

if __name__ == '__main__':
    wordc.run()


Writing wordcount.py


In [ ]:
%%writefile input.txt
Hello world
Hello Hadoop
MapReduce is fun
Hadoop is powerful


Writing input.txt


In [ ]:
!python wordcount.py input.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/wordcount.root.20250428.112252.596331
Running step 1 of 1...
job output is in /tmp/wordcount.root.20250428.112252.596331/output
Streaming final output from /tmp/wordcount.root.20250428.112252.596331/output...
"hello"	2
"is"	2
"mapreduce"	1
"powerful"	1
"fun"	1
"hadoop"	2
"world"	1
Removing temp directory /tmp/wordcount.root.20250428.112252.596331...


In [4]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
%%writefile avg_weather.py
from mrjob.job import MRJob

class MRAverageWeather(MRJob):

    def mapper(self, _, line):
        if line.startswith('Date'):  # Skip header
            return

        columns = line.split()
        if len(columns) >= 4:
            try:
                temperature = float(columns[1])
                dew_point = float(columns[2])
                wind_speed = float(columns[3])
                yield ("weather", (temperature, dew_point, wind_speed, 1))
            except ValueError:
                pass

    def reducer(self, key, values):
        total_temp = 0
        total_dew_point = 0
        total_wind_speed = 0
        count = 0

        for temperature, dew_point, wind_speed, _ in values:
            total_temp += temperature
            total_dew_point += dew_point
            total_wind_speed += wind_speed
            count += 1

        if count > 0:
            avg_temp = total_temp / count
            avg_dew_point = total_dew_point / count
            avg_wind_speed = total_wind_speed / count
            yield ("Average", (avg_temp, avg_dew_point, avg_wind_speed))
        else:
            yield ("Average", (None, None, None))

if __name__ == '__main__':
    MRAverageWeather.run()


Writing avg_weather.py


In [ ]:
%%writefile weather.txt
Date       Temperature DewPoint WindSpeed
2023-01-01 12.3         5.4      15
2023-01-02 14.2         6.1      18
2023-01-03 13.7         5.8      12
2023-01-04 15.0         7.0      20


Writing weather.txt


In [ ]:
!python avg_weather.py weather.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/avg_weather.root.20250428.112440.179302
Running step 1 of 1...
job output is in /tmp/avg_weather.root.20250428.112440.179302/output
Streaming final output from /tmp/avg_weather.root.20250428.112440.179302/output...
"Average"	[13.8, 6.075, 16.25]
Removing temp directory /tmp/avg_weather.root.20250428.112440.179302...


In [5]:
%%writefile log_processor.py
from mrjob.job import MRJob

class LogFileProcessor(MRJob):

    def mapper(self, _, line):
        try:
            # No decoding needed in Colab, line is already str
            parts = line.split(' ', 2)
            if len(parts) >= 2:
                log_level = parts[1]
                yield (log_level, 1)
            else:
                yield ("MALFORMED", 1)
        except Exception as e:
            yield ("ERROR", 1)

    def reducer(self, key, values):
        try:
            yield (key, sum(values))
        except Exception as e:
            yield ("ERROR", 1)

if __name__ == '__main__':
    LogFileProcessor.run()


Overwriting log_processor.py


In [6]:
%%writefile logs.txt
2025-04-28 10:00:00 INFO Starting system
2025-04-28 10:01:00 ERROR Failed to load module
2025-04-28 10:02:00 INFO System running
2025-04-28 10:03:00 WARNING Disk almost full
2025-04-28 10:04:00 ERROR Could not connect to server


Overwriting logs.txt


In [7]:
!python log_processor.py logs.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/log_processor.root.20250428.122125.885535
Running step 1 of 1...
job output is in /tmp/log_processor.root.20250428.122125.885535/output
Streaming final output from /tmp/log_processor.root.20250428.122125.885535/output...
"10:02:00"	1
"10:03:00"	1
"10:04:00"	1
"10:00:00"	1
"10:01:00"	1
Removing temp directory /tmp/log_processor.root.20250428.122125.885535...
